<a href="https://colab.research.google.com/github/jrojasquiroz/hppAI/blob/main/evacal/posible_correccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


En este cuaderno me dedico a corregir la base de datos original que le dimos a los chicos del Seminario (`v-muestra_rmb-abril24-responsables`). En ella hay observaciones que tienen pisos *okupados* y por tanto sin fotos.

Voy a reemplazarlas por otras ofertas.

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Verificación de enlaces

Dado que el corte inicial fue el 13 de mayo, hoy 22 de mayo volveré a revisar los enlaces inválidos para quitarlos de la base de datos original.

In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
from urllib.parse import urlparse
import geopandas as gpd
from shapely.geometry import Point
import re

In [ ]:
rmb=pd.read_excel('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/11-Web scrapping/data/primera_ronda/v-rmb-abril24.xlsx', sheet_name='Sheet1')
len(rmb)

24688

Le voy a hacer un filtro de precios, dado que he visto casos con `precio_euros` 0, 1 y de hasta 1800.

In [ ]:
rmb=rmb[rmb['precio_euros']>10000]
len(rmb)

24688

Antes de elegir una muestra, voy a eliminar todas aquellos casos que no tienen un enlace válido. El método que voy a usar no es perfecto para los enlaces válidos (es decir, se puede colar algún inválido), pero sí para los inválidos (todos son enlaces que ya no funcionan).

In [ ]:
# Función para verificar si un enlace es válido
def verificar_enlace_valido(link):
    try:
        # Realizamos una solicitud HEAD al enlace
        response = requests.head(link, headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'})

        # Verificar si el código de estado comienza con '3'
        if str(response.status_code).startswith('3'):
            return 1  # Enlace inválido
        else:
            return 0  # Enlace válido
    except:
        return 1  # En caso de algún error al abrir el enlace, lo marcamos como inválido

# Aplicamos la función a la columna 'Link' y guardamos los resultados en una nueva columna 'invalido'
rmb['invalido'] = rmb['Link'].apply(verificar_enlace_valido) #Con corte al 22 de mayo

rmb.to_excel('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/cuarto_intento/rawdata/1-v-rmb-abril24-invalidos-v2.xlsx', index=False)

Como he guardado el avance en `v-rmb-abril24-invalidos.xlsx`, vuelvo a llamar al archivo para no tener que repetir el proceso.

In [ ]:
rmb=pd.read_excel('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/cuarto_intento/rawdata/1-v-rmb-abril24-invalidos-v2.xlsx')
len(rmb)

24688

¿Cuántos son inválidos?

In [ ]:
len(rmb[rmb['invalido']==1])

57

Ahora el dataframe lo dejo sólo con las observaciones válidas.

In [ ]:
df=rmb[rmb['invalido']==0]
len(df)

24631

## Verificación de pisos *okupados*

Ya había hecho esto antes, pero ahora añado algunas palabras clave más.

In [ ]:
keywords = [
    'ocupado', 'piso ocupado', 'sin justo título', 'no admite visitas',
    'ocupa', 'okupa', 'no se puede ver interiormente', 'no se puede ver', 'con inquilinos',
    'ilegalmente', 'ilegal', 'situación ocupacional', 'posesión', 'procedimiento judicial', 'judicial',
    'judicialmente'
]

# Función para detectar palabras clave en la descripción
def detect_keywords(description):
    description_lower = description.lower()
    for keyword in keywords:
        # Usar una expresión regular para detectar la palabra clave
        if re.search(r'\b' + re.escape(keyword) + r'\b', description_lower):
            return 1
    return 0

# Crear la nueva columna 'prob_sin_foto' usando la función anterior
df['prob_sin_foto'] = df['Description'].apply(detect_keywords)

<ipython-input-9-78dd05086781>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prob_sin_foto'] = df['Description'].apply(detect_keywords)


In [ ]:
len(df[df['prob_sin_foto']==1])

1879

Y procedo a explorar los quintiles de ingreso.

In [ ]:
df=df[df['prob_sin_foto']==0]
df['qprecio'] = pd.qcut(df['precio_euros'], 5, labels=False)

<ipython-input-11-14a15170a0ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['qprecio'] = pd.qcut(df['precio_euros'], 5, labels=False)


In [ ]:
len(df)

22752

## Calculo quintiles a nivel general

Reviso la cantidad de casos por cada quintil.

In [ ]:
print(len(df[df['qprecio']==0]))
print(len(df[df['qprecio']==1]))
print(len(df[df['qprecio']==2]))
print(len(df[df['qprecio']==3]))
print(len(df[df['qprecio']==4]))

4599
4519
4574
4509
4551


Ahora voy a añadir el dataframe de municipios de España para que cada punto embeba el municipio al que pertenece.

In [ ]:
geomun=gpd.read_file('/content/drive/MyDrive/Datos/españa/vectores/MUNICIPOS-ESPAÑA-31N.gpkg')

# Convertir el DataFrame a un GeoDataFrame
geometry = [Point(lon, lat) for lon, lat in zip(df['Lon_X'], df['Lat_Y'])]
crs = 'epsg:4326'  # Definir sistema de coordenadas WGS84
gdf_puntos = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
gdf_puntos = gdf_puntos.to_crs(epsg=32631)

In [ ]:
print(geomun.crs)
print(gdf_puntos.crs)

EPSG:32631
EPSG:32631


In [ ]:
print(geomun.columns.tolist())

['INSPIREID', 'COUNTRY', 'NATLEV', 'NATLEVNAME', 'NATCODE', 'NAMEUNIT', 'CODNUT1', 'CODNUT2', 'CODNUT3', 'geometry']


In [ ]:
gdf_rmb = gpd.sjoin(gdf_puntos, geomun[['NAMEUNIT','geometry']], how='inner', predicate='intersects')
gdf_rmb=gdf_rmb.drop(['geometry','index_right'],axis=1)

len(gdf_rmb)

22718

In [ ]:
22752-22718

34

Hay 34 observaciones menos que en `df` debido a que esas no tenían información de su ubicación.

In [ ]:
print(gdf_rmb.columns.tolist())

['ID', 'Title', 'Link', 'precio_euros', 'codigo_inmueble', 'Description', 'last_mod', 'dummy_indice', 'indice_alquiler', 'area', 'n_rooms', 'price_m2', 'n_baths', 'n_aseo', 'laundry', 'buhardilla', 'despacho', 'trastero', 'terrace', 'kwhm2_year', 'kgCO2m2_year', 'dummy_consum_EPC', 'dummy_emission_EPC', 'consum_A', 'consum_B', 'consum_C', 'consum_D', 'consum_E', 'consum_F', 'consum_G', 'emission_A', 'emission_B', 'emission_C', 'emission_D', 'emission_E', 'emission_F', 'emission_G', 'AC', 'parking', 'pool', 'lift', 'furniture', 'public_transp', 'barrio', 'codigo_inmueble_numero', 'Lat_Y', 'Lon_X', 'multifam', 'atico', 'duplex', 'estudio', 'loft', 'piso', 'triplex', 'invalido', 'prob_sin_foto', 'qprecio', 'NAMEUNIT']


In [ ]:
gdf_rmb.rename(columns={'NAMEUNIT':'MUN'}, inplace=True)

`gdf_rmb` es el base sobre la cual se calcula la muestra; es decir, el universo de nuestros datos. La exporto para tenerla a la mano.

## Calculo quintiles por municipio

In [9]:
def calcular_quintiles_por_municipio(df):
    for municipio, df_municipio in df.groupby('MUN'):
        if len(df_municipio) > 10:
            # Calcula los quintiles para el municipio actual
            df.loc[df['MUN'] == municipio, 'qprecio_municipio'] = pd.qcut(df_municipio['precio_euros'], 5, labels=False, duplicates='drop')
        else:
            # Si el municipio tiene 10 o menos observaciones, asigna pd.NA a los quintiles
            df.loc[df['MUN'] == municipio, 'qprecio_municipio'] = pd.NA
    return df

# Aplica la función a tu DataFrame
gdf_rmb = calcular_quintiles_por_municipio(gdf_rmb)

In [10]:
gdf_rmb.to_excel('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/cuarto_intento/rawdata/2-v-universo_rmb-abril24-v2.xlsx', index=False)

# Revisión del dataframe

Si las columnas de calidad ( `C_contempo`, `C_estado`, ...) están vacías, revisar (1) que el link siga siendo válido y (2) que la descripción no haga referencia a un piso okupado.

Todos aquellos casos que no cumplan esas dos condiciones serán reemplazados por alguna otra observación del mismo `MUN` y del mismo `qprecio_municipio`.

In [15]:
muestra=pd.read_excel('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/cuarto_intento/data/v-muestra_rmb-abril24-responsables.xlsx')
print(muestra.columns.tolist())

['ID', 'Title', 'Link', 'precio_euros', 'codigo_inmueble', 'Description', 'last_mod', 'dummy_indice', 'indice_alquiler', 'area', 'n_rooms', 'price_m2', 'n_baths', 'n_aseo', 'laundry', 'buhardilla', 'despacho', 'trastero', 'terrace', 'kwhm2_year', 'kgCO2m2_year', 'dummy_consum_EPC', 'dummy_emission_EPC', 'consum_A', 'consum_B', 'consum_C', 'consum_D', 'consum_E', 'consum_F', 'consum_G', 'emission_A', 'emission_B', 'emission_C', 'emission_D', 'emission_E', 'emission_F', 'emission_G', 'AC', 'parking', 'pool', 'lift', 'furniture', 'public_transp', 'barrio', 'codigo_inmueble_numero', 'Lat_Y', 'Lon_X', 'multifam', 'atico', 'duplex', 'estudio', 'loft', 'piso', 'triplex', 'invalido', 'prob_sin_foto', 'qprecio', 'MUN', 'qprecio_municipio', 'C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano', 'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular', 'R_ventana', 'dumi_precio_reduccion', 'Precio_reduccion', 'Anuncio_invalido', 'Responsable', 'REV']


In [16]:
# Función para verificar si un enlace es válido
def verificar_enlace_valido(link):
    try:
        # Realizamos una solicitud HEAD al enlace
        response = requests.head(link, headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'})
        # Verificar si el código de estado comienza con '3'
        if str(response.status_code).startswith('3'):
            return 1  # Enlace inválido
        else:
            return 0  # Enlace válido
    except:
        return 1  # En caso de algún error al abrir el enlace, lo marcamos como inválido

# Columnas a verificar para NaNs
columnas_verificar = [
    'C_estado', 'C_armarios', 'B_contempo', 'B_estado',
    'B_lavamano', 'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular',
    'R_ventana', 'dumi_precio_reduccion', 'Precio_reduccion', 'Anuncio_invalido', 'REV'
]

# Filtrar los casos con NaNs en las columnas especificadas
casos_con_nans = muestra[muestra[columnas_verificar].isna().all(axis=1)]

# Aplicar la función de verificación de enlaces y actualizar la columna 'invalido'
muestra.loc[casos_con_nans.index, 'invalido'] = casos_con_nans.apply(lambda row: verificar_enlace_valido(row['Link']), axis=1)

# Asegurar que los valores de 'invalido' sean enteros
muestra['invalido'] = muestra['invalido'].astype(int)

Ojo: no estoy considerando `C_contempo` porque tiene relleno un espacio en blanco en todas las filas.

In [17]:
# Lista de palabras clave
keywords = [
    'ocupado', 'piso ocupado', 'sin justo título', 'no admite visitas',
    'ocupa', 'okupa', 'no se puede ver interiormente', 'no se puede ver', 'con inquilinos',
    'ilegalmente', 'ilegal', 'situación ocupacional', 'posesión', 'procedimiento judicial', 'judicial',
    'judicialmente'
]

# Función para detectar palabras clave en la descripción
def detect_keywords(description):
    if pd.isna(description):
        return 0
    description_lower = description.lower()
    for keyword in keywords:
        if re.search(r'\b' + re.escape(keyword) + r'\b', description_lower):
            return 1
    return 0

# Columnas a verificar para NaNs
columnas_verificar = [
    'C_estado', 'C_armarios', 'B_contempo', 'B_estado',
    'B_lavamano', 'R_contempo', 'R_estado', 'R_carpinteria', 'R_singular',
    'R_ventana', 'dumi_precio_reduccion', 'Precio_reduccion', 'Anuncio_invalido', 'REV'
]

# Filtrar los casos con NaNs en las columnas especificadas
casos_con_nans = muestra[muestra[columnas_verificar].isna().all(axis=1)]

# Aplicar la función detect_keywords y actualizar la columna 'prob_sin_foto'
muestra.loc[casos_con_nans.index, 'prob_sin_foto'] = casos_con_nans['Description'].apply(detect_keywords)

# Asegurar que los valores de 'prob_sin_foto' sean enteros
muestra['prob_sin_foto'] = muestra['prob_sin_foto'].astype(int)

In [18]:
len(muestra)

3456

In [19]:
muestra.to_excel('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/cuarto_intento/data/v-muestra_rmb-abril24-responsables-v2.xlsx',index=False)

Al final veo que sólo ha reconcido 35 casos nuevos como `prob_sin_foto`==1. Me parece que no vale la pena hacer el cambio de base de datos. Lo dejo aquí.

Importo `gdf_rmb` porque de ahí sacaré los datos para reemplazar en `muestra`.

In [8]:
gdf_rmb=pd.read_excel('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/cuarto_intento/rawdata/2-v-universo_rmb-abril24-v2.xlsx')